In [1]:
import numpy as np
import matplotlib.pyplot as plt
import keras

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
%matplotlib inline

In [3]:
from keras.preprocessing.image import ImageDataGenerator

In [4]:
from keras.applications.resnet50 import ResNet50, preprocess_input

In [5]:
from tqdm import tqdm

ModuleNotFoundError: No module named 'tqdm'

In [16]:
resnet = ResNet50(include_top=False, weights='imagenet')

D:\Anaconda3\envs\keras-gpu\lib\site-packages\keras_applications\resnet50.py:263: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


# Pre process images, transform in tensors

In [17]:
train_data_dir = 'data/train/'
validation_data_dir = 'data/valid/'
test_data_dir = 'data/test/'

In [74]:
img_height, img_width = 224, 224
batch_size = 100

In [6]:
train_datagen = ImageDataGenerator()
valid_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
train_data_dir, shuffle=False,
target_size = (img_height, img_width),
batch_size = batch_size, 
class_mode = "categorical")

validation_generator = valid_datagen.flow_from_directory(
validation_data_dir, shuffle=False,
target_size = (img_height, img_width),
class_mode = "categorical")

test_generator = test_datagen.flow_from_directory(
test_data_dir, shuffle=False,
target_size = (img_height, img_width),
class_mode = "categorical")

NameError: name 'train_data_dir' is not defined

In [247]:
train_features = resnet.predict_generator(train_generator, workers=16)
valid_features = resnet.predict_generator(validation_generator, workers=16)
test_features = resnet.predict_generator(test_generator, workers=16)

In [248]:
#one hot target
from keras.utils import to_categorical

In [249]:
train_labels = to_categorical(train_generator.classes)
valid_labels = to_categorical(validation_generator.classes)
test_labels = to_categorical(test_generator.classes)

In [250]:
test_labels

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

## Transfer Learning Architecture

In [251]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, GlobalAveragePooling2D

In [252]:
model = Sequential()

In [253]:
model.add(GlobalAveragePooling2D(input_shape=(7, 7, 2048)))
#model.add(Flatten(input_shape=(7, 7, 2048)))
model.add(Dropout(0.25))
model.add(Dense(20))
model.add(Dropout(0.25))
model.add(Dense(3, activation='softmax'))

In [254]:
model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [255]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_16  (None, 2048)              0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_24 (Dense)             (None, 20)                40980     
_________________________________________________________________
dropout_10 (Dropout)         (None, 20)                0         
_________________________________________________________________
dense_25 (Dense)             (None, 3)                 63        
Total params: 41,043
Trainable params: 41,043
Non-trainable params: 0
_________________________________________________________________


In [256]:
model.fit(train_features, train_labels, batch_size=256, epochs=100, 
          verbose=2, validation_data=(valid_features, valid_labels))

Train on 2000 samples, validate on 150 samples
Epoch 1/100
 - 3s - loss: 1.0328 - acc: 0.5985 - val_loss: 0.9871 - val_acc: 0.5400
Epoch 2/100
 - 1s - loss: 0.8292 - acc: 0.6660 - val_loss: 0.9507 - val_acc: 0.5400
Epoch 3/100
 - 1s - loss: 0.7192 - acc: 0.7020 - val_loss: 0.8969 - val_acc: 0.5933
Epoch 4/100
 - 1s - loss: 0.6976 - acc: 0.7135 - val_loss: 0.9268 - val_acc: 0.6200
Epoch 5/100
 - 1s - loss: 0.6716 - acc: 0.7240 - val_loss: 0.8986 - val_acc: 0.6267
Epoch 6/100
 - 1s - loss: 0.6381 - acc: 0.7305 - val_loss: 0.9676 - val_acc: 0.6267
Epoch 7/100
 - 1s - loss: 0.6389 - acc: 0.7325 - val_loss: 0.8332 - val_acc: 0.6667
Epoch 8/100
 - 1s - loss: 0.6064 - acc: 0.7545 - val_loss: 0.7985 - val_acc: 0.6267
Epoch 9/100
 - 1s - loss: 0.5763 - acc: 0.7635 - val_loss: 0.7716 - val_acc: 0.6267
Epoch 10/100
 - 1s - loss: 0.5648 - acc: 0.7595 - val_loss: 0.7897 - val_acc: 0.6600
Epoch 11/100
 - 1s - loss: 0.5447 - acc: 0.7780 - val_loss: 0.8280 - val_acc: 0.6800
Epoch 12/100
 - 1s - loss: 

Epoch 97/100
 - 1s - loss: 0.3357 - acc: 0.8600 - val_loss: 0.9289 - val_acc: 0.6667
Epoch 98/100
 - 1s - loss: 0.3193 - acc: 0.8695 - val_loss: 0.9918 - val_acc: 0.6667
Epoch 99/100
 - 1s - loss: 0.3324 - acc: 0.8550 - val_loss: 1.0470 - val_acc: 0.6600
Epoch 100/100
 - 1s - loss: 0.3245 - acc: 0.8660 - val_loss: 0.9166 - val_acc: 0.7000


In [258]:
valid_preds = model.predict(valid_features)

In [259]:
valid_preds

array([[2.14200411e-02, 9.60145295e-01, 1.84346084e-02],
       [1.30466267e-01, 1.47910535e-01, 7.21623182e-01],
       [2.72153765e-01, 6.73933983e-01, 5.39122559e-02],
       [7.46093038e-03, 9.90991354e-01, 1.54774857e-03],
       [4.74968851e-01, 5.18103838e-01, 6.92735054e-03],
       [2.78793484e-01, 7.07252979e-01, 1.39535507e-02],
       [2.36961190e-02, 5.81128597e-01, 3.95175308e-01],
       [7.54493833e-01, 1.07753336e-01, 1.37752831e-01],
       [3.06966066e-01, 4.45830449e-02, 6.48450971e-01],
       [4.10636567e-04, 9.98184860e-01, 1.40445458e-03],
       [7.20855128e-03, 3.96574765e-01, 5.96216679e-01],
       [2.57378012e-01, 7.00491607e-01, 4.21304218e-02],
       [3.85845780e-01, 5.66184819e-01, 4.79693972e-02],
       [4.37721550e-01, 3.44805717e-01, 2.17472687e-01],
       [9.17475484e-03, 2.65156537e-01, 7.25668669e-01],
       [2.32739486e-02, 3.63251790e-02, 9.40400839e-01],
       [9.14500296e-01, 3.71761844e-02, 4.83235046e-02],
       [8.39514434e-01, 1.55032

In [260]:
test_preds = model.predict(test_features)

In [261]:
import pandas as pd

In [262]:
df = pd.read_csv('sample_predictions.csv')

In [263]:
df.head()

,Id,task_1,task_2
0,data/test/melanoma/ISIC_0012258.jpg,0.0,0.0
1,data/test/melanoma/ISIC_0012356.jpg,0.0,0.0
2,data/test/melanoma/ISIC_0012369.jpg,0.0,0.0
3,data/test/melanoma/ISIC_0012395.jpg,0.0,0.0
4,data/test/melanoma/ISIC_0012425.jpg,0.0,0.0


In [264]:
df.shape

(600, 3)

In [265]:
test_preds.shape

(600, 3)

In [266]:
df['task_1'] = test_preds[:,0]
df['task_2'] = test_preds[:,2]

In [267]:
df.head()

,Id,task_1,task_2
0,data/test/melanoma/ISIC_0012258.jpg,0.115200,0.589082
1,data/test/melanoma/ISIC_0012356.jpg,0.136392,0.399431
2,data/test/melanoma/ISIC_0012369.jpg,0.518980,0.094591
3,data/test/melanoma/ISIC_0012395.jpg,0.093911,0.299976
4,data/test/melanoma/ISIC_0012425.jpg,0.001671,0.018164


In [268]:
df.to_csv('predictions_mine.csv', index=False)